In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

In [7]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_chinese import LocalFilesLoader, create_qa_chain

docs = LocalFilesLoader()
chunks = docs.load_and_split()

db = Chroma.from_documents(chunks, OpenAIEmbeddings(model="text-embedding-3-small"))

chain = create_qa_chain(ChatOpenAI(), db.as_retriever())

In [8]:
chain.invoke("供应商达标的业绩要求是什么")

AIMessage(content='供应商达标的业绩要求包括具备健全和稳定的财务状况、月销售额不得低于人民币3万元、产品必须符合国家及行业标准并通过相关质量认证、具有至少五年的相关行业经验、有良好的业务声誉和客户满意度记录。')

In [6]:
async for chunk in chain.astream_events("供应商达标的业绩要求是什么?", version="v1"):
	# print(chunk)
	# print(chunk['event'])
	if(chunk['event'] == "on_chat_model_stream"):
		print(chunk['data']['chunk'].content, end="|", flush=True)
	elif(chunk['event'] == "on_retriever_end"):
		for doc in chunk['data']['output']['documents']:
			print(doc.metadata['source'])


/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-chinese-gSQlHcwW-py3.9/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf
/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf
/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf
/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/求职简历.docx
|供|应|商|达|标|的|业|绩|要|求|是|每|个|月|销|售|的|产品|总|价|不|得|低|于|人|民|币|3|万元|。||